<a href="https://colab.research.google.com/github/alvumu/TGINE/blob/main/5_2_Entrenar_Word2vec_y_Doc2vec_desde_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 5 - Entrenar Word2vec y Doc2Vec desde 0

En este notebook vamos a ver cómo se entrena un modelo sencillo de word2vec eligiendo las dimensiones de los vectores.

Además, se creará un modelo de Doc2Vec a partir de un conjunto de documentos y se verá cómo se puede obtener las similitudes entre documentos.


In [1]:
# Instalamos gensim si no lo tenemos instalado
!pip3 install -U gensim
# Esto es por si no está ya instalado
!pip3 install -U pandas
!pip3 install -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.2 which is incompatible.


# Apartado 1.1 Descargamos un corpus de prueba

Vamos a probar con un corpus de noticias que se encuentra en la URL https://dis.um.es/~valencia/recursosTGINE/corpusNoticias.zip

In [2]:
# Descargamos un corpus de noticias que he creado
!wget https://dis.um.es/~valencia/recursosTGINE/corpusNoticias.zip
!unzip corpusNoticias.zip > extract.log

--2023-11-09 18:04:42--  https://dis.um.es/~valencia/recursosTGINE/corpusNoticias.zip
Resolving dis.um.es (dis.um.es)... 155.54.239.5
Connecting to dis.um.es (dis.um.es)|155.54.239.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4013278 (3.8M) [application/zip]
Saving to: ‘corpusNoticias.zip’

corpusNoticias.zip  100%[===================>]   3.83M   702KB/s    in 6.2s    

2023-11-09 18:04:49 (632 KB/s) - ‘corpusNoticias.zip’ saved [4013278/4013278]



Leemos todos lo ficheros y los metemos en una variable *texts*

Tened en cuenta que la codificación de caracteres en estos ficheros es ISO-8859-1. Esto depende del contenido de las web a descargar.

In [3]:
from os import listdir
from os.path import isfile, join

my_path = "corpusNoticias/"
texts = []
for fn in listdir(my_path):
  f = open(my_path+fn, encoding = "ISO-8859-1")
  file_content = f.read()
  texts.append(file_content)
  f.close()


# Apartado 1.2 Entrenamos un modelo word2vec a partir del corpus

Aquí vamos a entrenar un modelo word2vec con la librería GENSIM. Como Tokenizer se utilizará el word_tokenize de NLTK, pero se podría usar cualquier otro tokenizador.

In [4]:
from os import listdir
from os.path import isfile, join
import numpy
import pandas
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec,KeyedVectors
from gensim.test.utils import datapath
import re
import unicodedata
from tqdm import tqdm
import gensim
import multiprocessing
import random
from nltk.tokenize import word_tokenize


In [5]:
# Procesamos todos los textos y le aplicamos el word_tokenize de NLTK
nltk.download('punkt')
train_texts=[]
for text in texts:
     train_texts.append(word_tokenize(text.lower()))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from gensim.models import Word2Vec
# define training data
# train model
# se puede entrenar el modelo con distintos parámetros como el tamaño del vector, tamaño de la ventana, las veces que debe
# aparecer una palabra, etc.
model = Word2Vec(train_texts, vector_size=100, window=10, min_count=1, workers=10)
# summarize the loaded model
print(model)
# save model
model.save('model.bin')


Word2Vec<vocab=65297, vector_size=100, alpha=0.025>


In [10]:
# Cargamos el modelo guardado
new_model = Word2Vec.load('model.bin')

# Probamos el nuevo modelo
# Imprimimos el vector de la palabra 'energía'
print(model.wv['covid'])


[-2.3171072  -0.6353457  -0.8882953   1.2679191   0.35203254 -1.8359014
  2.445024    0.13145125 -1.6552632   0.12183941  0.22062442  0.9824246
 -0.5657579  -1.1925262  -1.2993544   0.570515    0.6292578   1.0638626
 -0.44025388 -1.3687452  -0.32334003  1.3650624   1.9366636  -1.9743267
  0.7758547  -1.0670822  -1.1597335   1.9702872  -2.5840924   0.72169995
  0.90783095  0.11759519  0.44998652 -0.72499347 -1.7860962   1.3648189
  0.29275566  1.2438548   0.56664103 -1.9164493  -0.45516044 -1.7284216
  0.24511221  0.7505731   0.9222006   1.2025623  -1.1068743   0.08298312
  0.5174444  -0.43226063  0.09337439 -0.5965135   1.4815034   0.24663968
 -1.7978597  -0.59846014  0.7590351  -1.3042297  -0.64423674  0.74606025
 -0.79167914 -2.271411    1.086545   -0.5687606  -0.59196526  1.636033
 -0.4013893   1.4139998  -0.6512565  -1.6435883   0.5724434   1.031522
  0.70069915  0.15508503  0.8327893   0.9024068   0.8017701  -0.10283418
  1.3944242   1.1426065  -0.86106753 -0.9127982   0.9626993  

In [38]:
# Probamos similitudes
new_model.wv.similarity("vox", "pp")

0.9203681

In [39]:
# Probamos en listar alguna de las palabras más similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'energía'
palabra = 'murcia'
print(new_model.wv.most_similar(palabra))

[('covid-19', 0.9676380753517151), ('coronavirus', 0.9658496379852295), ('contagio', 0.9258149862289429), ('sars-cov-2', 0.8964800238609314), ('inmunización', 0.8935132026672363), ('astrazeneca', 0.8776937127113342), ('pfizer', 0.8774365782737732), ('datos', 0.8624992370605469), ('vacunación', 0.8622227907180786), ('contagios', 0.8578690886497498)]
[('universidad', 0.9598396420478821), ('extremadura', 0.9273892641067505), ('mancha', 0.9203846454620361), ('aragón', 0.9197322130203247), ('instituto', 0.918502151966095), ('umu', 0.9170052409172058), ('castilla-la', 0.9164246916770935), ('emergencias', 0.9148464202880859), ('león', 0.9112562537193298), ('johns', 0.9102033972740173)]


In [40]:
# Probamos alguna analogía
# Covid es a Vacunas lo que Salud es a ...
print(new_model.wv.most_similar(positive=["salud", "vacunas"], negative=["covid"], topn=10))

# Covid es a Vacunas lo que Guerra es a ...
print(new_model.wv.most_similar(positive=["guerra", "vacunas"], negative=["covid"], topn=10))


[('pública', 0.8432562947273254), ('medidas', 0.8076187372207642), ('ema', 0.7950373291969299), ('seguridad', 0.7827852368354797), ('restrictivas', 0.7800939083099365), ('epidemiológicos', 0.7764226794242859), ('variantes', 0.7759642004966736), ('gotzone', 0.7687575817108154), ('prevención', 0.7626656293869019), ('sanidad', 0.7590991854667664)]
[('respuesta', 0.8536342978477478), ('divulgativa', 0.8428986072540283), ('estrategia', 0.8390644788742065), ('normativa', 0.8372287750244141), ('económica', 0.8295809626579285), ('voluntad', 0.8258192539215088), ('justa', 0.8228832483291626), ('importancia', 0.8226196765899658), ('necesidad', 0.8223133683204651), ('normalización', 0.8219084143638611)]


In [41]:
# Probamos a mostrar términos similares
# Imprimimos las palabras más similares a 'covid'
palabra = 'covid'
print(new_model.wv.most_similar(palabra))

# Imprimimos las palabras más similares a 'ucrania'
palabra = 'ucrania'
print(new_model.wv.most_similar(palabra))

[('covid-19', 0.9676380753517151), ('coronavirus', 0.9658496379852295), ('contagio', 0.9258149862289429), ('sars-cov-2', 0.8964800238609314), ('inmunización', 0.8935132026672363), ('astrazeneca', 0.8776937127113342), ('pfizer', 0.8774365782737732), ('datos', 0.8624992370605469), ('vacunación', 0.8622227907180786), ('contagios', 0.8578690886497498)]
[('guerra', 0.885240912437439), ('urgente', 0.8849346041679382), ('conjunta', 0.8787731528282166), ('personal', 0.8767902255058289), ('actual', 0.8765659928321838), ('urgencia', 0.8745409846305847), ('invasión', 0.867993175983429), ('rusia', 0.8656632900238037), ('económica', 0.8603065013885498), ('inmigración', 0.8587011098861694)]


# Apartado 1.3 Entrenamos un Doc2Vec con los mismos textos


In [42]:
#Import all the dependencies
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

#Necestiamos crear un TaggedDocument para cada uno de los textos indicando un índice de cada texto
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(train_texts)]


In [43]:
# Definimos los parámetros de entrenamiento y entrenamos
max_epochs = 10
vec_size = 50
alpha = 0.025

doc2vec_model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm = 1,
                epochs = max_epochs)

doc2vec_model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    doc2vec_model.train(tagged_data,
                total_examples=doc2vec_model.corpus_count,
                epochs=doc2vec_model.epochs)
    # decrease the learning rate
    doc2vec_model.alpha -= 0.0002
    # fix the learning rate, no decay
    doc2vec_model.min_alpha = model.alpha

doc2vec_model.save("d2v.model")
print("Model Saved")

Model Saved


In [44]:
from gensim.models import Doc2Vec

doc2vec_model= Doc2Vec.load("d2v.model")
# Probamos a encontrar textos similares a uno dado
query_text = "gas energía"
test_data = word_tokenize(query_text)
v1 = doc2vec_model.infer_vector(test_data)

# Encontramos los documentos más similares
similar_doc = doc2vec_model.dv.most_similar(v1)

# Imprimimos los 5 documentos más similares
top5_similar_doc = similar_doc[:5]
print(top5_similar_doc)
for doc in top5_similar_doc:
  print("--------------------------")
  print(texts[int(doc[0])])
  print('Similitud:',doc[1])



[('685', 0.6811789274215698), ('649', 0.6720859408378601), ('1399', 0.6513968706130981), ('753', 0.6513261795043945), ('1418', 0.6493618488311768)]
--------------------------
Un Mobile World Congress con cifras de recuperación económica
GSMA se da por satisfecha. La patronal organizadora del Mobile World Congress presentó ayer un balance que cumple las expectativas marcadas al inicio del Salón, que ayer cerró sus puertas. 60.000 personas pisaron el recinto Gran Via de Fira de Barcelona durante las cuatro jornadas del gran encuentro anual de la industria de las telecomunicaciones, una cifra que sus responsables habían situado en la zona alta de su tabla de objetivos. Presencialidad es la palabra que podría resumir el balance de esta edición, que llegaba tras un Mobile de transición como el que se desarrolló el pasado mes de junio, cuando 20.000 personas asistieron a un congreso que descargó gran parte de su programación en el formato virtual para sortear los obstáculos derivados de la p